In [1]:
import pandas as pd
import re
from functools import partial
from ourtypes.category import Category
from ourtypes.award import Award
import pickle 
import numpy
from rank_bm25 import BM25Okapi
import spacy
"""with open("common_words_hosts.pickle", "rb") as pckl:   
    common_words_host_list = pickle.load(pckl)
with open("common_words_wins.pickle", "rb") as pckl:
    common_words_winners_list = pickle.load(pckl)"""

'with open("common_words_hosts.pickle", "rb") as pckl:   \n    common_words_host_list = pickle.load(pckl)\nwith open("common_words_wins.pickle", "rb") as pckl:\n    common_words_winners_list = pickle.load(pckl)'

Set up tweets df

In [2]:
preprocessed_tweets = "gg2013_preprocessed.json"
tweets = pd.read_json(preprocessed_tweets, orient='records', lines=True)[["text","is_english"]]
tweets = tweets[tweets["is_english"]] #only keep english tweets
del tweets["is_english"] #get rid of this row
tweets = tweets.dropna(subset=["text"])

In [3]:
#get hosts for entire ceremony
#get award names for the ceremony

In [4]:
def remove_common_words(text, word_list):
    #negative positive golden globes
    words = [word for word in text.split(" ") if word not in word_list]
    return " ".join(words)

In [5]:
def remove_rt(text):
    text = text.split(" ")
    
    if "rt" in text:
        i = text.index("rt")
        return remove_rt(" ".join(text[0:i]) + " " + " ".join(text[i+2:]))
    return " ".join(text)

function to get hosts

In [6]:
def hosts_helper(text, host_funcs, cohost_funcs, single_cohost_funcs, hosts):
    #check for cohosts first
    for func in cohost_funcs:
        m = func.search(text)
        if m is not None:
            name1 = remove_common_words(m["name1"], common_words_host_list)
            name2 = remove_common_words(m["name2"], common_words_host_list)
            hosts.vote_contender(name1, cocontender=name2)
            hosts.vote_contender(name2, cocontender=name1)
            return name1, name2
    #check for hosts
    for func in host_funcs:
        m = func.search(text)
        if m is not None:
            name = remove_common_words(m["name"], common_words_host_list)
            hosts.vote_contender(name)
            return name
    for func in single_cohost_funcs:
        m = func.search(text)
        if m is not None:
            name = remove_common_words(m["name"], common_words_host_list)
            hosts.vote_contender(name)
            return name
    return None

In [7]:
def get_hosts(tweets):
    hosts = Category(type="hosts") #our host storage object
    host_tweets = tweets[tweets["text"].str.contains("host")] #get tweets with the word host in it
    #del host_tweets["timestamp"] #we don't need this 
    host_tweets = host_tweets.map(remove_rt) #get rid of retweets
    #handle single names or no spaces?
    host_patterns = ['host (?P<name>[a-z]+ [a-z]+)', '(?P<name>[a-z]+ [a-z]+) (hosting|is hosting|will host|hosts|hosted)']
    cohost_patterns = ['(co-?|)hosts (?P<name1>[a-z]+ [a-z]+)( and | )(?P<name2>[a-z]+ [a-z]+)',
                    '(?P<name1>[a-z]+ [a-z]+)( and | )(?P<name2>[a-z]+ [a-z]+) (are (co-?|)hosting|will (co-?|)host|(co-?|)host|(co-?|)hosting|hosted)[^s]']
    single_cohost_patterns = ['cohost (?P<name>[a-z]+ [a-z]+)',
                    '(?P<name>[a-z]+ [a-z]+) (is cohosting|will cohost|cohosts|cohosting|cohosted)[^s]']
    #compile regex functions
    host_funcs = [re.compile(pat) for pat in host_patterns]
    cohost_funcs = [re.compile(pat) for pat in cohost_patterns]
    single_cohost_funcs = [re.compile(pat) for pat in single_cohost_patterns]

    #apply helper function 
    host_tweets = host_tweets.map(partial(hosts_helper, host_funcs=host_funcs, cohost_funcs=cohost_funcs, single_cohost_funcs=single_cohost_funcs, hosts=hosts))
    del host_tweets
    return hosts

In [8]:
#h = get_hosts(tweets)
#vc = h.total_votes()
#match misspelling and single name 

## Part Two
- have awards as input
- get presenters for awards
- get nominees for awards
- get winners for awards

In [9]:
awards_list = ["best performance by an actor in a television series - comedy or musical", 
          "best performance by an actor in a television series - drama",
          "best performance by an actor in a motion picture - drama",
          "best performance by an actress in a mini-series or motion picture made for television",
          "best original song - motion picture", 
          "best animated feature film",
"best television series - comedy or musical",
          "best performance by an actor in a mini-series or motion picture made for television", 
          "best television series - drama", 
          "best performance by an actress in a supporting role in a motion picture",
          "best performance by an actor in a supporting role in a series, mini-series or motion picture made for television", 
          "best motion picture - drama",
          "best performance by an actor in a motion picture - comedy or musical", 
          "cecil b. demille award", 
          "best performance by an actress in a motion picture - drama",
          "best performance by an actress in a television series - drama", 
          "best original score - motion picture", 
          "best mini-series or motion picture made for television",
          "best performance by an actress in a motion picture - comedy or musical", 
          "best motion picture - comedy or musical", 
          "best performance by an actress in a supporting role in a series, mini-series or motion picture made for television",
          "best performance by an actor in a supporting role in a motion picture",
          "best foreign language film", 
          "best performance by an actress in a television series - comedy or musical",
          "best director - motion picture", "best screenplay - motion picture"]

"best performance by an actress in a mini-series or motion picture made for television",
"best original song - motion picture", 
          "best animated feature film",
"best television series - comedy or musical",
          "best performance by an actor in a mini-series or motion picture made for television", 
          "best television series - drama", 
          "best performance by an actress in a supporting role in a motion picture",
          "best performance by an actor in a supporting role in a series, mini-series or motion picture made for television", 
          "best motion picture - drama",
          "best performance by an actor in a motion picture - comedy or musical", 
          "cecil b. demille award", 
          "best performance by an actress in a motion picture - drama",
          "best performance by an actress in a television series - drama", 
          "best original score - motion picture", 
          "best mini-series or motion picture made for television",
          "best performance by an actress in a motion picture - comedy or musical", 
          "best motion picture - comedy or musical", 
          "best performance by an actress in a supporting role in a series, mini-series or motion picture made for television",
          "best performance by an actor in a supporting role in a motion picture",
          "best foreign language film", 
          "best performance by an actress in a television series - comedy or musical",
          "best director - motion picture", "best screenplay - motion picture"]

Format awards 

In [10]:
def format_awards(awards): 
    results = {}
    person_patterns = "actor|cecil|actress|direction|director|choreo|perfomance"
    thing_patterns = "series|mini-series|movie|film|music|comedy|song|score|motion picture"
    person_func = re.compile(person_patterns)
    thing_func = re.compile(thing_patterns)
    for award in awards:
        m = person_func.search(award)
        if m is not None:
            results[award] = Award(award, winner_type="Person")
            continue
        m = thing_func.search(award)
        if m is not None:
            results[award] = Award(award, winner_type="Thing")
        else:
            results[award] = Award(award)
    return results
        

In [11]:
def award_aliases(key, val):
    #remove performance by an
    #television -> tv
    #television series -> tv
    #remove feature
    #in a supporting role - supporting actress/actor 
    #film -> movie
    #motion picture -> movie
    key = key.replace(".", "")
    #key = key.replace("made for television", "tv")
    #key = key.replace("movie", "movie film")
    aliases = [key]
    if "performance by an" in key:
        aliases += list(map(lambda x: x.replace("performance by an", ""), aliases))
    if "foreign language" in key:
        aliases += list(map(lambda x: x.replace("foreign language", "foreign"), aliases))
    if "television series" in key:
        aliases += list(map(lambda x: x.replace("television series", "tv"), aliases))
    if "television" in key:
        aliases += list(map(lambda x: x.replace("television", "tv"), aliases))
    if "motion picture" in key:
        aliases += list(map(lambda x: x.replace("motion picture", "movie"), aliases))
    if "feature" in key:
        aliases += list(map(lambda x: x.replace("feature", ""), aliases))
    if "original" in key:
        aliases += list(map(lambda x: x.replace("original", ""), aliases))
    
    if "actor in a supporting role" in key:
        aliases += list(map(lambda x: x.replace("actor in a supporting role", "supporting actor"),aliases))
    elif "actress in a supporting role" in key:
        aliases += list(map(lambda x: x.replace("actress in a supporting role", "supporting actress"), aliases))
    if "motion picture made for television" in key:
        aliases += list(map(lambda x: x.replace("motion picture made for television", "tv movie"),aliases))
    if "director" in key:
        aliases += list(map(lambda x: x[:x.find("-")], aliases))

    aliases += list(map(lambda x: x.replace("-", ""), aliases))
    aliases = list(map(lambda x: x.replace("  ", " "), aliases))
    
    val.add_alias(aliases)

In [12]:
def awards_struct(awards_list):
    awards = format_awards(awards_list)
    for k, v in awards.items():
        award_aliases(k,v)
    return awards

### Winners

add bm25 search as well

In [13]:
tweets = tweets.map(remove_rt)
corpus = tweets.to_numpy()
tokenized_corpus = [str(tweet).split(" ") for tweet in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [14]:
spacy_model = spacy.load("en_core_web_sm")

In [15]:
def bm25_search(award_name):
    q = award_name.split(" ")
    scores = bm25.get_scores(q)
    indices = numpy.argsort(scores)
    relevant = tweets.iloc[indices]
    return relevant[-500:]

In [16]:
def winner_stop_words(tweet1, list2=False):
    tweet = str(tweet1)
    toreplace = ["goldenglobes", "anclerts", "award for", "just", "goes to", "mr president", "love him", "for winning",
                 "has known", "finally", "yes", "at the", " is ", "first", " for ", "bazinga rs", "amen",
                 "no surpres", "well deserved", "no surprises", "this generation", " to ", "goldenglobe", "goldenglobes"]
    if list2: toreplace = ["golden", "live", "blog", "globes", "annual", "award for", "goldenglobes", "award", "awards"]
    for ele in toreplace:
        tweet = tweet.replace(ele, " ")
    
    if "http" in tweet:
        tweetList = tweet.split(" ")
        for ele in tweetList:
            if "http" in ele:
                tweetList.remove(ele)
        tweet = " ".join(tweetList)
    if tweet == tweet1:
        tweet = tweet.replace(".", "").replace(" - "," ")
        return tweet.replace("  "," ").replace("  "," ")
    else:
        return winner_stop_words(tweet)

In [17]:
def winner_helper(tweet, v, won_funcs):
    if v.winner_type == "Person":
        tweet = winner_stop_words(tweet)
    else:
        tweet = winner_stop_words(tweet, list2=True)
        
    for f in won_funcs:
        m = f.findall(tweet)
        if m != []:
            if isinstance(m[0], tuple):
                for ele in m[0]:
                    if v.winner_type != "Person":
                        i = ele.find("by")
                        if i != -1:
                            ele = ele[:i]
                    else:
                        if "win" in ele or "won" in ele:
                            continue

                    v.add_winner(ele)
                    v.add_nominee(ele)
                    if v.winner_type != "Person": break
            else:
                ele = m[0]
                if v.winner_type != "Person":
                    i = ele.find("by")
                    if i != -1:
                        ele = ele[:i]
                else:
                    if "win" in ele or "won" in ele:
                        continue
                v.add_winner(ele)
                v.add_nominee(ele) 
                if v.winner_type != "Person": break
    return

In [18]:
def extra_winner_helper(tweet, v):
    spacy_output = spacy_model(tweet)
    for entity in spacy_output.ents:
        if entity.label_ == "PERSON":
            v.add_winner(entity.text)
            v.add_nominee(entity.text)
    return

In [19]:
awards = awards_struct(awards_list)
for k, v in awards.items():
    
    #print(k)
    award_names = v.aliases
    won_patterns = [f"{award} (?P<name>[a-z]+ ?[a-z-]+)" for award in award_names]
    won_patterns += [f"(?P<name>[a-z]+ ?[a-z-]+)( wins? | on winning | has? won | got| wins the | ){award}" for award in award_names]
    won_funcs = [re.compile(ele) for ele in won_patterns]
    #presenter_patterns = [f"(?P<name>[a-z]+ ?[a-z]+)( presents? | presenting | are presenting ){award}" for award in award_names]
    #presenter_funcs = [re.compile(ele) for ele in presenter_patterns]
    relevant = bm25_search(award_names[0])
    if v.winner_type == "Person":
        relevant = relevant.map(partial(winner_helper, v=v, won_funcs=won_funcs))
        if v.winners.contenders == {}:
            relevant = bm25_search(award_names[-1])
            relevant = relevant.map(partial(extra_winner_helper, v=v))
    else:
        won_patterns = [f"{award} [for ]?(?P<name>[a-z ]+)" for award in award_names]
        won_patterns += [f"(?P<name>[a-z ]+)( wins? | on winning | has? won | got| wins the | ){award}" for award in award_names]
        won_funcs = [re.compile(ele) for ele in won_patterns]
        relevant = relevant.map(partial(winner_helper, v=v, won_funcs=won_funcs))
        
    print(str(v))
    #print(str(v.nominees))
"""if v.winner_type == "Person":
    spacy_output = spacy_model(tweet)
    for entity in spacy_output.ents:
        if entity.label_ == "PERSON":
            v.add_winner(entity.text)
            v.add_nominee(entity.text)"""

"""for f in presenter_funcs:
    m = f.findall(tweet)
    if m != []:
        print(tweet)
        if isinstance(m[0], tuple):
            for ele in m[0]:
                if ele == " ": continue
                v.add_presenter(ele)
        else:
            v.add_presenter(m[0])"""

Award: best performance by an actor in a television series - comedy or musical
Nominees: Top: don cheadle, Votes: 14
Winners: Top: don cheadle, Votes: 14
Presenters: 

Award: best performance by an actor in a television series - drama
Nominees: Top: damian lewis, Votes: 34
Winners: Top: damian lewis, Votes: 34
Presenters: 

Award: best performance by an actor in a motion picture - drama
Nominees: Top: daniel day-lewis, Votes: 43
Winners: Top: daniel day-lewis, Votes: 43
Presenters: 

Award: best performance by an actress in a mini-series or motion picture made for television
Nominees: Top: julianne moore, Votes: 7
Winners: Top: julianne moore, Votes: 7
Presenters: 

Award: best original song - motion picture
Nominees: Top: skyfall, Votes: 126
Winners: Top: skyfall, Votes: 126
Presenters: 

Award: best animated feature film
Nominees: Top: brave, Votes: 104
Winners: Top: brave, Votes: 104
Presenters: 

Award: best television series - comedy or musical
Nominees: Top: girls, Votes: 45.5
Wi

'for f in presenter_funcs:\n    m = f.findall(tweet)\n    if m != []:\n        print(tweet)\n        if isinstance(m[0], tuple):\n            for ele in m[0]:\n                if ele == " ": continue\n                v.add_presenter(ele)\n        else:\n            v.add_presenter(m[0])'

In [20]:
print(awards["best foreign language film"].nominees)

Name: lmbooo, Votes: 4.5
Name: lmbooo we, Votes: 6.0
Name: lmbooo we were, Votes: 5.5
Name: lmbooo we were on, Votes: 6.0
Name: we were, Votes: 2
Name: we were on the, Votes: 17.5
Name: were on, Votes: 2
Name: were on the same, Votes: 6.5
Name: on the, Votes: 2
Name: on the same page, Votes: 4.0
Name: the same, Votes: 2
Name: same page, Votes: 2
Name: presented, Votes: 11.0
Name: amour, Votes: 60
Name: amour no, Votes: 3.0
Name: amour no surprise, Votes: 2.0
Name: amour no surprise here, Votes: 2.0
Name: no surprise, Votes: 1
Name: no surprise here all, Votes: 2.5
Name: surprise here, Votes: 1
Name: surprise here all the, Votes: 1.5
Name: here all, Votes: 1
Name: here all the nominees, Votes: 4.5
Name: all the, Votes: 1
Name: all the nominees were, Votes: 1.5
Name: the nominees, Votes: 3
Name: the nominees were axing, Votes: 2.0
Name: nominees were, Votes: 1
Name: nominees were axing films, Votes: 4.0
Name: were axing, Votes: 1
Name: were axing films so, Votes: 2.0
Name: axing films, V

In [21]:
for v in awards.values():
    print(str(v))

Award: best performance by an actor in a television series - comedy or musical
Nominees: Top: don cheadle, Votes: 14
Winners: Top: don cheadle, Votes: 14
Presenters: 

Award: best performance by an actor in a television series - drama
Nominees: Top: damian lewis, Votes: 34
Winners: Top: damian lewis, Votes: 34
Presenters: 

Award: best performance by an actor in a motion picture - drama
Nominees: Top: daniel day-lewis, Votes: 43
Winners: Top: daniel day-lewis, Votes: 43
Presenters: 

Award: best performance by an actress in a mini-series or motion picture made for television
Nominees: Top: julianne moore, Votes: 7
Winners: Top: julianne moore, Votes: 7
Presenters: 

Award: best original song - motion picture
Nominees: Top: skyfall, Votes: 126
Winners: Top: skyfall, Votes: 126
Presenters: 

Award: best animated feature film
Nominees: Top: brave, Votes: 104
Winners: Top: brave, Votes: 104
Presenters: 

Award: best television series - comedy or musical
Nominees: Top: girls, Votes: 45.5
Wi

input_sentence = "kerry washington at the  golden globes wearing vintage movado kw will be presenting at the awards"
spacy_output = spacy_model(input_sentence)
for chunk in spacy_output.noun_chunks:
    print([chunk.text, chunk.root.head.text])
    print("\n")


for award in awards.keys():
    relevant = bm25_search(award)
    won_patterns = [f"{award}( goes to | is | )(?P<name>[a-z]+ ?[a-z]+)", f"(?P<name>[a-z]+ ?[a-z]+)[just]?( wins? | on winning | won | has[ just]? won | got| )[ award for ]?{award}"]
    won_funcs = [re.compile(ele) for ele in won_patterns]
    #presenter_patterns = [f"(?P<name>[a-z]+ ?[a-z]+)( presents | is presenting | are presenting | present )[ award for ]?"]
    #presenter_funcs = [re.compile(ele) for ele in presenter_patterns]
    for tweet in relevant:
        for f in won_funcs:
            tweet = str(tweet)
            m = f.search(tweet)
            if m is not None:
                name = m["name"]
                if awards[award].winner_type =="Person":
                    print(name)
                    #name = remove_common_words(name, common_words_winners_list)
                awards[award].add_winner(name)
        """for f in presenter_funcs:
            tweet = str(tweet)
            m = f.search(tweet)
            if m is not None:
                name = remove_common_words(m["name"], common_words_winners_list)
                name = m["name"]
                awards[award].add_presenter(name)"""

In [22]:
"""l = []
with open("common_words_winner_nominees_presenters.txt", "w+") as f:
    for k in awards.keys():
        a = awards[k].winners
        c = a.contenders
        for v in c.values():
            for name in v.name.split(" "):
                if name not in l:
                    f.write(name + "\n")
                    l.append(name)"""

'l = []\nwith open("common_words_winner_nominees_presenters.txt", "w+") as f:\n    for k in awards.keys():\n        a = awards[k].winners\n        c = a.contenders\n        for v in c.values():\n            for name in v.name.split(" "):\n                if name not in l:\n                    f.write(name + "\n")\n                    l.append(name)'

In [23]:
"""with open("common_words_wins.pickle", "wb") as f:
    pickle.dump(l, f)
"""

'with open("common_words_wins.pickle", "wb") as f:\n    pickle.dump(l, f)\n'

## Get Award Names

Function to get awards

def clean_and_sort_text(text):
    # function to group award names, even if they are missing a key word or a "-"
    regex_pattern =  r"(motion+| original+|)"
    text = text.replace(regex_pattern, ' ')
    words = text.replace("-", ' ').split()
    # sort words and join back into a string
    return ' '.join(sorted(words))


def extract_award_names(df, text_column='text'):
    extracted_award_names = []
    # indentify which tweets might contain an award name
    keywords = ['wins', 'awarded', 'goes to']
    pattern = '|'.join(keywords)
    candidate_tweets = df[df[text_column].str.contains(pattern, regex=True)]

    # matching regex to all the tweets
    regex_pattern = r"(wins|awarded|goes to)(.*?)(award)?($|[,.!?:;])"
    for tweet in candidate_tweets[text_column]:
        matches = re.findall(regex_pattern, tweet)
        for match in matches:
            award_name = match[1].strip()
            if award_name:
                extracted_award_names.append(award_name)

    # clean up the results so we can rank them
    delimiters = r"( http+| for +| at +| goldenglobes +| odds-on +| and +| goes +)"
    cleaned_award_names = [re.split(delimiters, name, 1)[0] for name in extracted_award_names]
    cleaned_award_names = [name for name in cleaned_award_names if name.lower().startswith('best') and len(name.split()) > 1]
    cleaned_award_df = pd.DataFrame(cleaned_award_names, columns=['Cleaned_Award_Name'])
    cleaned_award_counts = cleaned_award_df['Cleaned_Award_Name'].value_counts().reset_index()
    cleaned_award_counts.columns = ['Cleaned_Award_Name', 'Frequency']

    cleaned_award_counts['Sorted_Award_Name'] = cleaned_award_counts['Cleaned_Award_Name'].apply(clean_and_sort_text)
    
    # chooses the longest award name of the grouping
    def prefer_longer(series):
        return max(series, key=len)

    # group by the cleaned and sorted award names and add the frequencies together
    grouped_award_counts = cleaned_award_counts.groupby('Sorted_Award_Name').agg({
        'Cleaned_Award_Name': prefer_longer,  # keep the longest version
        'Frequency': 'sum'  # add up their frequencies
    }).reset_index(drop=True)

    grouped_award_counts = grouped_award_counts[grouped_award_counts['Frequency'] > 1]
    grouped_award_counts = grouped_award_counts.sort_values(by='Frequency', ascending=False).reset_index(drop=True)
    return grouped_award_counts

#award_names = extract_award_names(tweets)
#print(award_names)
